In [ ]:
# Advanced Workflow Patterns

This notebook demonstrates advanced patterns and best practices:
- Dynamic workflow generation
- Conditional branching and complex control flow
- Event-driven workflows
- Integration with external systems
- Performance optimization techniques


In [ ]:
import os
from kubiya_workflow_sdk import workflow, step
from kubiya_workflow_sdk.dsl import when, retry_policy
import json

api_key = os.getenv("KUBIYA_API_KEY", "test-key")
print("✅ SDK loaded for advanced patterns")


In [ ]:
# Example 1: Dynamic Workflow Generation
def create_deployment_workflow(
    app_name: str, 
    environments: list, 
    run_tests: bool = True,
    notify_slack: bool = True
):
    """Dynamically generate a deployment workflow based on parameters."""
    
    wf = workflow(f"deploy-{app_name}")
    wf.description(f"Deploy {app_name} to {len(environments)} environments")
    
    # Add build step
    wf.step("build", f"docker build -t {app_name}:latest .")
    
    # Conditionally add test step
    if run_tests:
        test_step = step("run-tests", f"docker run {app_name}:latest npm test")
        test_step.retry(2, 30)  # Retry tests up to 2 times
        wf.data["steps"].append(test_step.to_dict())
    
    # Deploy to each environment
    for env in environments:
        deploy_step = step(f"deploy-{env}", f"kubectl apply -f k8s/{env}/")
        
        # Add environment-specific configurations
        if env == "production":
            deploy_step.preconditions("${TEST_RESULT} == 'passed'")
            deploy_step.timeout(600)  # 10 minute timeout for prod
        
        wf.data["steps"].append(deploy_step.to_dict())
    
    # Add notification step if enabled
    if notify_slack:
        wf.step(
            "notify-slack",
            f"curl -X POST $SLACK_WEBHOOK -d '{\"text\":\"Deployed {app_name}!\"}'"
        )
    
    return wf

# Generate workflows for different scenarios
dev_workflow = create_deployment_workflow(
    "myapp", 
    ["dev"], 
    run_tests=False,
    notify_slack=False
)

prod_workflow = create_deployment_workflow(
    "myapp",
    ["staging", "production"],
    run_tests=True,
    notify_slack=True
)

print("📋 Dynamically generated workflows:")
print(f"   Dev workflow: {len(dev_workflow.data['steps'])} steps")
print(f"   Prod workflow: {len(prod_workflow.data['steps'])} steps")
